# Electronic DataSheet - Ontological Model Language

## Description
Welcome to our solution designed to streamline and standardize the use of Electronic Data Sheets (EDS) in the space industry. Our project focuses on enhancing the efficiency and accuracy of spacecraft design and integration by leveraging the power of EDS.
We choice to offer to our reader a better perspective of our datasheet by create a transformation typed model-to-text. To offer a readable description, we choice to show each element by the concatenation of its id and its name.

## Devices and metadatas
A datasheet includes a variety of elements such as devices, packages, components, interfaces, state machines, activities, and data types. In this context, we choice to provide a visualization highlighting all devices and their corresponding metadata.

Devices are centrale to the datasheet and form the system's core. Moreover, metadata provides essential information like configuration details, manufactuer data, operational modesl, and design-adjustables parameters, etc. So, they offer clear insights into the system. 

We've chosen to present a tree graph showcasing devices and their associated data, proving useful for visualizing system architecture, understanding device-metadata relationships, and aiding in efficient design decision-making.

In [75]:
from utilities import *

df = dataframe("device_metadata.json")
data = df.to_json(orient = "records")
HTML(tree.safe_substitute(data=data))

##

## Components Set

We choice to visualize how components are organized in a set.

In [97]:
from utilities import *
df = dataframe("componentSet.json")
componentSets = dict()
for index, row in df.iterrows():
    component_set = row['component_set']
    component_id = row['component_id']
    if component_set not in componentSets:
        componentSets[component_set] = [component_id]
    else:
        componentSets[component_set].append(component_id)

components = todict(df, 'component_id', 'component_name')

diagram_str = 'left to right direction\n'

for component_id,component_name in components.items():
    diagram_str += f'object "{component_id}: {component_name}" as {component_id}\n'
diagram_str += f'\n'
for key,values in componentSets.items():
    package_label = component_set.split('#')[-1] if '#' in component_set else component_set
    diagram_str += f'package "{package_label}" {{\n'
    # for value in set(values):
    #     diagram_str += f'   {value}\n'
    diagram_str += '}\n'

print(diagram_str)

diagram(diagram_str)


left to right direction
object "C.02: Physical category" as C.02
object "C.01: LED" as C.01
object "C.03: Lamp shade" as C.03

package "ComponentSet" {
}



![Alt text](http://www.plantuml.com/plantuml/img/JSen3i8m40NHtgVuyW4G8LrQeAD0uWGRPx4DYTUojuaGTySKaTADz6Rcfq859VgWc69Xfr6IaV7L2lPyE9u6tCDQewCPZfIzbDM2Alws7xi1j-jbesvFVIDQCcgWYRUZDoQJUvFdziYI9N7I1wl5ntpD3m==)

In [108]:
from utilities import *
df = dataframe("componentSet.json")

# Initialize a dictionary to store component sets and their components
componentSets = dict()

# Populate the componentSets dictionary
for _, row in df.iterrows():
    component_set = row['component_set']
    component_id = row['component_id']
    component_name = row['component_name']
    component_label = f"{component_id}: {component_name}"

    if component_set not in componentSets:
        componentSets[component_set] = [(component_id,component_name)]
    else:
        componentSets[component_set].append((component_id,component_name))

# Start building the diagram string
diagram_str = '@startuml\nleft to right direction\n'

# Define each component set as a package and include its components
for component_set, component_labels in componentSets.items():
    # Extract a readable label from the component set URI
    package_label = component_set.split('#')[-1] if '#' in component_set else component_set
    diagram_str += f'package "Component set : {package_label}" {{\n'
    for component_label in set(component_labels):
        # Define the component as an object inside the package
        diagram_str += f'    object "{component_label[0]}: {component_label[1]}" as {component_label[0]}\n'
    diagram_str += '}\n'

diagram_str += '@enduml'

# Generate the diagram
diagram(diagram_str)


![Alt text](http://www.plantuml.com/plantuml/img/NOux4i8m34HxdsB71s18w5nb-7HqEO68HMBmRsnHCTmTKuGYwbRlIRD35Iho2bvvlWiaeRXv4LXNU1ANeieqFMbcw5CAEKMEWie2WtyUMJJU2ctIxT7EchlR7mok53BgGfOrgEAtt5fTioxd5NTRt1iSgJ9y-xHAlVgeWQDjfRy=)

In [ ]:
from utilities import *
df = dataframe("components.json")
df = df.rename(columns={"deviceId" : "device id" , "deviceName": "device name" , "componentId" : "component id" , "componentName" : "component name" , "interfaceId" : "interface id", "interfaceName" : "interface name", "metadataId" : "metadata id", "metadataName" : "metadata name"})
style = df.style.hide(axis="index").set_properties(**{'text-align': 'left', 'font-size': '12pt',})
style.format(precision=2).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

component_set_id,component_set_name,component_id,component_name
CS.01,Lamp component set,C.01,Electrical category
CS.01,Lamp component set,C.01,LED
CS.01,Lamp component set,C.01,Electrical category
CS.01,Lamp component set,C.01,LED
CS.01,Lamp component set,C.02,Base lamp
CS.01,Lamp component set,C.02,Physical category
CS.01,Lamp component set,C.02,Base lamp
CS.01,Lamp component set,C.02,Physical category
CS.01,Lamp component set,C.03,Lamp shade


## Components and interfaces

To increase our understand the need and have of our syste, we've developped a visualization that retrieves each component's ID and name, along with the names and IDs of their associated interfaces—those they provide, require, and declare.

To do that, we choice to create a component diagram. It effectively outlines the components and their interfaces. Each component's unique identifiers and names are displayed alongside the interfaces they interact with—categorized as provided, required, or declared. This representation is crucial for understanding the system's design, highlighting which interfaces are needed.

**Context:**
- **Components** in our system are entities with computational behavior and interfaces, forming part of a ComponentSet.
- **Provided Interface** represents functionalities that the component offers to other system parts.
- **Required Interfaces** are functionalities that the component needs from other parts of the system.
- **Declared Interfaces** are specific to a component, used internally and not accessible externally.

In [ ]:
from utilities import *
df = dataframe("component_interface.json")

components = todict(df, 'component_id', 'component_name')
interfaces_required = todict(df, 'interface_required_id', 'interface_required_name') if 'interface_required_id' in df.columns else {}
interfaces_provided = todict(df, 'interface_provided_id', 'interface_provided_name') if 'interface_provided_id' in df.columns else {}
interfaces_declared = todict(df, 'interface_declared_id', 'interface_declared_name') if 'interface_declared_id' in df.columns else {}

compositions = []

if 'interface_required_id' in df.columns and 'component_id' in df.columns:
    compositions += [(row['component_id'], row['interface_required_id'], "required")
                     for _, row in df.iterrows() if pd.notna(row['interface_required_id'])]

if 'interface_provided_id' in df.columns and 'component_id' in df.columns:
    compositions += [(row['component_id'], row['interface_provided_id'], "provided")
                     for _, row in df.iterrows() if pd.notna(row['interface_provided_id'])]

if 'interface_declared_id' in df.columns and 'component_id' in df.columns:
    compositions += [(row['component_id'], row['interface_declared_id'], "declared")
                     for _, row in df.iterrows() if pd.notna(row['interface_declared_id'])]

compositions = set(compositions)

diagram_str = 'left to right direction\nskinparam componentStyle uml2 10\n'

for key,value in {**components}.items():
    diagram_str += f'component "{key}: {value}" as {key}\n'
for key,value in {**interfaces_required, **interfaces_provided, **interfaces_declared}.items():
    diagram_str += f'interface "{key}: {value}" as {key}\n'

for src, dest, label in compositions:
    diagram_str += f'{dest} --> {src} : {label}\n'

diagram_str += '@enduml'

diagram(diagram_str)


![Alt text](http://www.plantuml.com/plantuml/img/TP71QiCm38RlUOgVt3kQv9R3A5jxA6nGs65dOQkrgMC7M-d8syy9TAnfUp78tySl2Ja-2YGWsfCHQ1jPYGs-I6VhUuhKGOMk3vwzVCde64FdQbJhukyRvVlBkchniTkMe8IfMj2wnS6CoIfoK2Hy2d6yklN2RN8ITJsI8SrNgIci5uv7KenoFqjlRc09GGpIZnLbPdL_h-R9FiaFkVEjLosyFEDttjvnIWkpBgO7gzNhl1jQz35Sh6OzWUePQ9w1coZDob5y4FKVD0_1XhtEr_W5)